## deltaLens Data Compare Sample

In [1]:
#imports

from delta_lens.deltaLens import DeltaLens, EntityComparer
from delta_lens.config import Config, Entity, Side, Transform, Defaults,ReferenceDataset, load_config
from delta_lens.sqliteExport import *
from datetime import date

In [ ]:
#config

config = load_config("data/compare.config.json")


: 

In [ ]:
runName = f'tradeCompare_{date.today().strftime("%Y-%m-%d")}'

comparator = DeltaLens(runName,config)

comparator.execute(continue_on_error=True)



#### entity_compare_results
Table containing summary of each entity comparison

In [ ]:
comparator.con.execute("SELECT * FROM  entity_compare_results").fetch_df()

,entity,rows_left,rows_right,rows_fully_matched,error_text,success
0,trade,42999117,43000000,42113850,None,1


#### [entitiy]_compare_field_summary
Entity specific table containing summary of field comparison

In [ ]:
comparator.con.execute("SELECT * FROM trade_compare_field_summary").fetch_df()

,field,total,matches,match_percentage
0,trade_type,42999117,42999117.0,100.000000
1,quantity,42999117,42999117.0,100.000000
2,price,42999117,42113850.0,97.940002
3,timestamp,42999117,42999117.0,100.000000
4,symbol,42999117,42999117.0,100.000000


#### [entity]_compare
Entity specific table containing details of comparison

In [ ]:
comparator.con.execute(" WITH CTE as (SELECT * FROM  trade_compare WHERE _full_match = 0) SELECT * FROM CTE USING SAMPLE 5;").fetch_df()

,trade_id,trade_type_left,trade_type_right,trade_type_match,quantity_left,quantity_right,quantity_match,price_left,price_right,price_match,timestamp_left,timestamp_right,timestamp_match,symbol_left,symbol_right,symbol_match,_exists_left,_exists_right,_full_match
0,1347595,BUY,BUY,True,67,67,True,516.82,146.48,False,2024-06-12 20:34:12,2024-06-12 20:34:12,True,BAC,BAC,True,True,True,False
1,11688365,BUY,BUY,True,7613,7613,True,720.50,492.27,False,2024-04-16 20:35:28,2024-04-16 20:35:28,True,TSLA,TSLA,True,True,True,False
2,16660286,BUY,BUY,True,2974,2974,True,717.42,196.39,False,2024-06-28 20:36:04,2024-06-28 20:36:04,True,TSLA,TSLA,True,True,True,False
3,21232731,BUY,BUY,True,594,594,True,19.40,390.79,False,2024-04-17 20:36:38,2024-04-17 20:36:38,True,NVDA,NVDA,True,True,True,False
4,25537902,BUY,BUY,True,2989,2989,True,314.71,340.96,False,2024-03-24 20:37:10,2024-03-24 20:37:10,True,AAPL,AAPL,True,True,True,False


#### Export to sqlite
Function to export above tables to on-disk format. Sqlite is a popular single file database, well supported in many languages,  for which lots of query tools exist. See [Datasette](https://datasette.io/) package.

In [ ]:
export_to_sqlite(comparator.con, f'{runName}.sqlite')

'tradeCompare_2025-02-13.sqlite'